In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.callbacks import EarlyStopping

2024-08-22 13:20:43.995152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 13:20:44.071002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-22 13:20:44.071021: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-22 13:20:44.431520: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
target_X= pd.read_csv("../data/solor_train_input_7.csv").iloc[:,(1+24*0):].values
target_y =pd.read_csv("../data/solor_train_output_7.csv").iloc[:,1:].values
test_X= pd.read_csv("../data/solor_val_input_7.csv").iloc[:,(1+24*0):].values
test_y =pd.read_csv("../data/solor_val_output_7.csv").iloc[:,1:].values

X_train = target_X
y_train = target_y
X_train.shape,y_train.shape

((721, 168), (721, 24))

In [4]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],1,y_train.shape[1],y_train.shape[1]

#################################################################################
# nbeats 모델 생성 함수
def build_model(input_timesteps,features,output_timesteps,unit):
    model = Sequential()
    model.add(LSTM(unit, return_sequences=True, input_shape=(input_timesteps, features)))
    #model.add(LSTM(unit, return_sequences=True))
    # Use Lambda layer to select the last 'output_timesteps' outputs
    model.add(Lambda(lambda x: x[:, -output_timesteps:, :]))
    model.add(TimeDistributed(Dense(1)))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    input_timesteps,features,output_timesteps,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_model(input_timesteps,features,output_timesteps,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
# SMAPE 용
def train_bagging_models_smape(num_models, loss_fn , epochs_, patience_,batch_size_):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim,unit)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys



#################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [5]:
model_num = 10


mase_models = train_bagging_models(model_num, MASE(y_train,24),100,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',100,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),100,10,8,0.001)

2024-08-22 13:21:31.950509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-22 13:21:31.950553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-08-22 13:21:31.951142: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
72/72 [==============================] - 2s 16ms/step - loss: 1.5916 - val_loss: 1.0696
Epoch 2/100
72/72 [==============================] - 1s 13ms/step - loss: 1.0592 - val_loss: 0.7495
Epoch 3/100
72/72 [==============================] - 1s 13ms/step - loss: 0.8505 - val_loss: 0.6526
Epoch 4/100
72/72 [==============================] - 1s 13ms/step - loss: 0.8161 - val_loss: 0.6472
Epoch 5/100
72/72 [==============================] - 1s 13ms/step - loss: 0.8058 - val_loss: 0.6303
Epoch 6/100
72/72 [==============================] - 1s 13ms/step - loss: 0.7939 - val_loss: 0.6405
Epoch 7/100
72/72 [==============================] - 1s 13ms/step - loss: 0.7849 - val_loss: 0.6150
Epoch 8/100
72/72 [==============================] - 1s 13ms/step - loss: 0.7749 - val_loss: 0.6286
Epoch 9/100
72/72 [==============================] - 1s 13ms/step - loss: 0.7739 - val_loss: 0.6302
Epoch 10/100
72/72 [==============================] - 1s 13ms/step - loss: 0.7711 - val_loss: 0.5935

In [6]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models

smape_predictions = bagging_predict2(pred1, test_X)
mase_predictions = bagging_predict2(pred2, test_X)
mape_predictions = bagging_predict2(pred3, test_X)
concat = np.concatenate([smape_predictions, mase_predictions,mape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
mean_squared_error(test_y.flatten(),fin_pred.flatten()),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

12/12 [==============================] - 0s 8ms/step


(0.23775922444365957, 0.2417236099818133)

In [7]:
pd.DataFrame(fin_pred.reshape(-1,24)).to_csv("../result7_new/LSTM/pred_mid.csv")
for i in range(10):
    pd.DataFrame(concat[i].reshape(-1,24)).to_csv(f"../result7_new/LSTM/pred{i}.csv")